
# K-Means Clustering — Math, Implementation, and Visualization

**Author:** Generated by ChatGPT  
**Date:** 2025-10-09 13:22

This notebook is a complete, practical guide to K-Means clustering. It covers:
- The math behind K-Means (objective, derivations, convergence)
- A from-scratch NumPy implementation (including k-means++)
- Usage with `scikit-learn`
- How to choose *k* (Elbow, Silhouette)
- Preprocessing (scaling), dimensionality reduction (PCA) for visualization
- Real and synthetic datasets (Iris, blobs, moons)
- Strengths, limitations, and practical tips

> All plots are made with **matplotlib** (no seaborn). Each chart is on its own figure.



## 1. Math Background

### 1.1 Problem Setup

Given data points \( X = \{x_1, x_2, \dots, x_n\} \subset \mathbb{R}^d \) and an integer \(k\), K-Means seeks cluster assignments \(c_i \in \{1,\dots,k\}\) and centroids \( \mu_1, \dots, \mu_k \in \mathbb{R}^d \) that minimize the **within-cluster sum of squares (WCSS)**:
\[
J(\{\mu_j\}, \{c_i\}) \;=\; \sum_{i=1}^{n} \lVert x_i - \mu_{c_i}\rVert_2^2.
\]

### 1.2 Alternating Minimization

K-Means uses **coordinate descent** (a.k.a. Lloyd’s algorithm):

1. **Assignment step (E-step):** with fixed centroids, assign each point to the nearest centroid  
\[
c_i \leftarrow \arg\min_{j \in \{1,\dots,k\}} \lVert x_i - \mu_j\rVert_2^2.
\]
2. **Update step (M-step):** with fixed assignments, update each centroid to the mean of its assigned points  
\[
\mu_j \leftarrow \frac{1}{|C_j|} \sum_{i: c_i = j} x_i, \quad C_j=\{i \mid c_i=j\}.
\]

**Derivation of the centroid update.** For fixed assignments, the objective decomposes across clusters:  
\[
J = \sum_{j=1}^k \sum_{i \in C_j} \lVert x_i - \mu_j \rVert_2^2.
\]
For a single cluster \(j\), minimize \( f(\mu_j) = \sum_{i \in C_j} \lVert x_i - \mu_j \rVert_2^2 \) w.r.t. \(\mu_j\). Taking derivative and setting to zero:  
\[
\frac{\partial f}{\partial \mu_j} = \sum_{i \in C_j} 2(\mu_j - x_i) = 2\left(|C_j|\mu_j - \sum_{i \in C_j} x_i\right)=0
\;\;\Rightarrow\;\;
\mu_j = \frac{1}{|C_j|}\sum_{i\in C_j} x_i.
\]

Thus, the update is the **cluster mean**.

### 1.3 Convergence & Complexity

- Each iteration **does not increase** \(J\) and the algorithm converges to a **local minimum** (not necessarily the global minimum).  
- Complexity per iteration is \(O(nkd)\): computing distances from \(n\) points to \(k\) centroids in \(d\) dimensions, plus \(O(nd)\) to recompute means.  
- Initialization matters (to avoid poor local minima)—**k-means++** is a standard choice.

### 1.4 Choosing \(k\)

- **Elbow method:** plot \(J(k)\) vs. \(k\) and look for an “elbow.”  
- **Silhouette score:** for sample \(i\), let \(a(i)\) be the mean intra-cluster distance and \(b(i)\) the mean nearest-cluster distance; the silhouette is  
\[
s(i) = \frac{b(i)-a(i)}{\max\{a(i), b(i)\}} \in [-1, 1].
\]
Higher is better; average over all points.

### 1.5 When K-Means Works / Fails

- Works best when clusters are **spherical**, **similar size**, and **similar density**.  
- Can fail on **non-convex** shapes (e.g., moons) or **varying density/size** clusters. Dimensionality reduction, feature scaling, or using other clustering methods may help.


In [ ]:

# Imports used across the notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import make_blobs, make_moons, load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

np.random.seed(42)
print("Versions: numpy", np.__version__)



## 2. From-Scratch K-Means (NumPy)

Below is a simple, educational implementation of K-Means. It supports:
- random initialization or k-means++
- maximum iterations and tolerance-based early stopping
- returning inertia (WCSS) and labels


In [ ]:

def pairwise_sq_dists(X, C):
    """
    Compute squared Euclidean distances between each row of X (n,d)
    and each row of C (k,d). Returns (n,k).
    """
    # ||x - c||^2 = ||x||^2 + ||c||^2 - 2 x.c
    X_norm = np.sum(X**2, axis=1, keepdims=True)     # (n,1)
    C_norm = np.sum(C**2, axis=1, keepdims=True).T   # (1,k)
    return X_norm + C_norm - 2 * X @ C.T

def init_centroids_random(X, k, rng=None):
    rng = np.random.default_rng(None if rng is None else rng)
    idx = rng.choice(X.shape[0], size=k, replace=False)
    return X[idx].copy()

def init_centroids_kmeans_plus_plus(X, k, rng=None):
    """
    Probabilistic initialization:
    - Pick first centroid at random
    - Next centroids chosen with probability proportional to D(x)^2 (distance to nearest chosen centroid)
    """
    rng = np.random.default_rng(None if rng is None else rng)
    n = X.shape[0]
    centroids = []
    # Choose first centroid randomly
    first = rng.integers(0, n)
    centroids.append(X[first])
    # Choose remaining
    for _ in range(1, k):
        d2 = np.min(pairwise_sq_dists(X, np.array(centroids)), axis=1)
        probs = d2 / np.sum(d2)
        next_idx = rng.choice(n, p=probs)
        centroids.append(X[next_idx])
    return np.array(centroids)

def kmeans_numpy(X, k, init="k-means++", max_iter=300, tol=1e-4, rng=None, verbose=False):
    """
    Simple K-Means using NumPy.
    Returns: centroids (k,d), labels (n,), inertia (float), n_iter (int)
    """
    X = np.asarray(X)
    if init == "k-means++":
        C = init_centroids_kmeans_plus_plus(X, k, rng=rng)
    elif init == "random":
        C = init_centroids_random(X, k, rng=rng)
    else:
        raise ValueError("init must be 'k-means++' or 'random'")

    prev_inertia = None
    for it in range(1, max_iter+1):
        # E-step: assign
        dists = pairwise_sq_dists(X, C)  # (n,k)
        labels = np.argmin(dists, axis=1)

        # M-step: update
        new_C = np.zeros_like(C)
        for j in range(k):
            mask = labels == j
            if np.any(mask):
                new_C[j] = X[mask].mean(axis=0)
            else:
                # Handle empty cluster: reinitialize to a random point
                new_C[j] = X[np.random.randint(0, X.shape[0])]

        # Check convergence
        inertia = np.sum((X - new_C[labels])**2)
        if verbose:
            print(f"Iter {it:3d} | inertia = {inertia:.4f}")
        if prev_inertia is not None and abs(prev_inertia - inertia) <= tol * prev_inertia:
            C = new_C
            break

        C = new_C
        prev_inertia = inertia

    return C, labels, inertia, it



## 3. Synthetic Data Demo (Blobs)

We create 2D Gaussian blobs and run both our NumPy K-Means and `scikit-learn`'s KMeans. We visualize the clusters and centroids.


In [ ]:

# Create synthetic blobs
X, y_true = make_blobs(n_samples=600, centers=4, cluster_std=1.10, random_state=42)

# Run our NumPy K-Means
C_np, labels_np, inertia_np, n_iter_np = kmeans_numpy(X, k=4, init="k-means++", max_iter=200, tol=1e-4, rng=42)
print("NumPy K-Means -> inertia:", inertia_np, "iterations:", n_iter_np)

# Plot result
plt.figure()
plt.scatter(X[:,0], X[:,1], s=12)
plt.scatter(C_np[:,0], C_np[:,1], marker="X", s=200)
plt.title("From-Scratch K-Means (NumPy) — Clusters and Centroids")
plt.xlabel("x1"); plt.ylabel("x2")
plt.show()

# Compare with scikit-learn
km = KMeans(n_clusters=4, n_init=10, random_state=42, init="k-means++")
km.fit(X)
print("sklearn KMeans -> inertia:", km.inertia_, "iterations:", km.n_iter_)

plt.figure()
plt.scatter(X[:,0], X[:,1], s=12)
plt.scatter(km.cluster_centers_[:,0], km.cluster_centers_[:,1], marker="X", s=200)
plt.title("scikit-learn KMeans — Clusters and Centroids")
plt.xlabel("x1"); plt.ylabel("x2")
plt.show()



## 4. Choosing \(k\): Elbow and Silhouette

We compute WCSS (inertia) for a range of \(k\) values to plot an elbow curve, and compute the average silhouette score to gauge cluster separation quality.


In [ ]:

Ks = range(2, 11)
inertias = []
sil_scores = []

for k in Ks:
    km = KMeans(n_clusters=k, n_init=10, random_state=42, init="k-means++")
    labels = km.fit_predict(X)
    inertias.append(km.inertia_)
    sil = silhouette_score(X, labels)
    sil_scores.append(sil)

# Elbow plot
plt.figure()
plt.plot(list(Ks), inertias, marker="o")
plt.title("Elbow Method (WCSS vs k)")
plt.xlabel("k"); plt.ylabel("WCSS (Inertia)")
plt.show()

# Silhouette plot
plt.figure()
plt.plot(list(Ks), sil_scores, marker="o")
plt.title("Average Silhouette Score vs k")
plt.xlabel("k"); plt.ylabel("Silhouette Score")
plt.show()



## 5. Preprocessing & 2D Visualization via PCA

Clustering often benefits from **feature scaling**. Here we scale features with `StandardScaler` and then reduce to 2D using `PCA` for visualization.


In [ ]:

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

km = KMeans(n_clusters=4, n_init=10, random_state=42)
labels = km.fit_predict(X_pca)

plt.figure()
plt.scatter(X_pca[:,0], X_pca[:,1], s=12)
plt.title("PCA-Reduced Data Colored by K-Means Labels")
plt.xlabel("PC1"); plt.ylabel("PC2")
plt.show()



## 6. Real Dataset: Iris

We cluster the classic Iris dataset into \(k=3\) clusters and visualize in PCA space, then compare cluster labels with true species (for reference only; K-Means is unsupervised).


In [ ]:

iris = load_iris()
X_iris = iris.data
species = iris.target

scaler_iris = StandardScaler()
X_iris_scaled = scaler_iris.fit_transform(X_iris)

pca_iris = PCA(n_components=2, random_state=42)
X_iris_pca = pca_iris.fit_transform(X_iris_scaled)

km_iris = KMeans(n_clusters=3, n_init=20, random_state=42, init="k-means++")
labels_iris = km_iris.fit_predict(X_iris_scaled)

plt.figure()
plt.scatter(X_iris_pca[:,0], X_iris_pca[:,1], s=20)
plt.title("Iris — PCA Projection with K-Means Labels")
plt.xlabel("PC1"); plt.ylabel("PC2")
plt.show()

# Quick evaluation via silhouette (unsupervised)
sil_iris = silhouette_score(X_iris_scaled, labels_iris)
print("Iris silhouette score (k=3):", round(sil_iris, 3))

# Optional: contingency table (unsupervised sanity-check)
import pandas as pd
df_ct = pd.crosstab(pd.Series(species, name="True"), pd.Series(labels_iris, name="Cluster"))
df_ct



## 7. Failure Case: Non-Convex Shapes (Two Moons)

K-Means assumes roughly spherical clusters. On a **two moons** dataset, it struggles to separate the shapes, unlike density-based alternatives. This section demonstrates the limitation.


In [ ]:

X_moons, y_moons = make_moons(n_samples=600, noise=0.08, random_state=42)

km_moons = KMeans(n_clusters=2, n_init=10, random_state=42)
labels_moons = km_moons.fit_predict(X_moons)

plt.figure()
plt.scatter(X_moons[:,0], X_moons[:,1], s=12)
plt.title("Two Moons — K-Means Result (Often Suboptimal)")
plt.xlabel("x1"); plt.ylabel("x2")
plt.show()

print("Silhouette score (moons, k=2):", round(silhouette_score(X_moons, labels_moons), 3))



## 8. Practical Tips

- **Scale features** when dimensions have different units/magnitudes.  
- Use **k-means++** initialization and **multiple restarts** (`n_init`) to reduce sensitivity to initialization.  
- Run **Elbow** and **Silhouette** to guide your choice of \(k\).  
- Consider **PCA** (or other DR) to mitigate noise and to visualize high-dimensional data.  
- Beware of **outliers**: K-Means uses squared distances, so outliers can distort centroids.  
- If your data has **non-convex** or **varying-density** clusters, consider DBSCAN or spectral clustering instead.



## 9. Appendix: K-Means Pseudocode

```
Input: data X ∈ ℝ^{n×d}, number of clusters k
Initialize centroids μ₁,…,μ_k  (e.g., k-means++)

Repeat until convergence or max_iter:
    # Assignment
    For i = 1..n:
        c_i ← argmin_j ||x_i − μ_j||²

    # Update
    For j = 1..k:
        μ_j ← mean({ x_i : c_i = j })
Output: centroids {μ_j}, labels {c_i}, inertia J = ∑_i ||x_i − μ_{c_i}||²
```
